# PDBe API Training

### PDBe search

Searching with a sequence

In [1]:
from pprint import pprint # used for pretty printing
import sys
sys.path.insert(0,'..') # to ensure the below import works in all Jupyter notebooks
from python_modules.api_modules import run_sequence_search, pandas_dataset, pandas_count, pandas_plot, pandas_plot_multi_groupby

c:\program files (x86)\python37-32\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


We will search for a sequence with an example sequence from UniProt P24941 -
Cyclin-dependent kinase 2

In [4]:
sequence_to_search = """
MENFQKVEKIGEGTYGVVYKARNKLTGEVVALKKIRLDTETEGVPSTAIREISLLKELNH
PNIVKLLDVIHTENKLYLVFEFLHQDLKKFMDASALTGIPLPLIKSYLFQLLQGLAFCHS
HRVLHRDLKPQNLLINTEGAIKLADFGLARAFGVPVRTYTHEVVTLWYRAPEILLGCKYY
STAVDIWSLGCIFAEMVTRRALFPGDSEIDQLFRIFRTLGTPDEVVWPGVTSMPDYKPSF
PKWARQDFSKVVPPLDEDGRSLLSQMLHYDPNKRISAKAALAHPFFQDVTKPVPHLRL"""

filter_list = ['pfam_accession', 'pdb_id', 'molecule_name', 'ec_number',
               'uniprot_accession_best', 'tax_id']

first_results = run_sequence_search(sequence_to_search, filter_terms=filter_list)

https://www.ebi.ac.uk/pdbe/search/pdb/select?group=true&group.field=pdb_id&group.ngroups=true&json.nl=map&start=0&sort=fasta(e_value) asc&xjoin_fasta=true&bf=fasta(percentIdentity)&xjoin_fasta.external.expupperlim=0.1&xjoin_fasta.external.sequence=
MENFQKVEKIGEGTYGVVYKARNKLTGEVVALKKIRLDTETEGVPSTAIREISLLKELNH
PNIVKLLDVIHTENKLYLVFEFLHQDLKKFMDASALTGIPLPLIKSYLFQLLQGLAFCHS
HRVLHRDLKPQNLLINTEGAIKLADFGLARAFGVPVRTYTHEVVTLWYRAPEILLGCKYY
STAVDIWSLGCIFAEMVTRRALFPGDSEIDQLFRIFRTLGTPDEVVWPGVTSMPDYKPSF
PKWARQDFSKVVPPLDEDGRSLLSQMLHYDPNKRISAKAALAHPFFQDVTKPVPHLRL&q=*:*&fq={!xjoin}xjoin_fasta&fl=pfam_accession,pdb_id,molecule_name,ec_number,uniprot_accession_best,tax_id&wt=json&rows=10
Number of results 10


Print the first result to see what we have

In [5]:
pprint(first_results[0])

{'e_value': 9.3e-77,
 'ec_number': ['2.7.11.22'],
 'molecule_name': ['Cyclin-dependent kinase 2'],
 'pdb_id': '2r3p',
 'percentage_identity': 100.0,
 'pfam_accession': ['PF00069'],
 'tax_id': [9606],
 'uniprot_accession_best': ['P24941']}


Before we do any further analysis we should get a few more results so we can see some patterns.
We are going to increase the number of results to 1000

In [6]:
first_results = run_sequence_search(sequence_to_search,
                                    filter_terms=filter_list,
                                    number_of_rows=1000
                                    )


https://www.ebi.ac.uk/pdbe/search/pdb/select?group=true&group.field=pdb_id&group.ngroups=true&json.nl=map&start=0&sort=fasta(e_value) asc&xjoin_fasta=true&bf=fasta(percentIdentity)&xjoin_fasta.external.expupperlim=0.1&xjoin_fasta.external.sequence=
MENFQKVEKIGEGTYGVVYKARNKLTGEVVALKKIRLDTETEGVPSTAIREISLLKELNH
PNIVKLLDVIHTENKLYLVFEFLHQDLKKFMDASALTGIPLPLIKSYLFQLLQGLAFCHS
HRVLHRDLKPQNLLINTEGAIKLADFGLARAFGVPVRTYTHEVVTLWYRAPEILLGCKYY
STAVDIWSLGCIFAEMVTRRALFPGDSEIDQLFRIFRTLGTPDEVVWPGVTSMPDYKPSF
PKWARQDFSKVVPPLDEDGRSLLSQMLHYDPNKRISAKAALAHPFFQDVTKPVPHLRL&q=*:*&fq={!xjoin}xjoin_fasta&fl=pfam_accession,pdb_id,molecule_name,ec_number,uniprot_accession_best,tax_id&wt=json&rows=1000
Number of results 819


Load the results into a Pandas Dataframe so we can query them

In [7]:
df = pandas_dataset(first_results)

Lets see what we have - you'll see it looks a bit like a spreadsheet or a database

In [8]:
print(df.head())

   ec_number              molecule_name pdb_id   pfam_accession tax_id  \
0  2.7.11.22  Cyclin-dependent kinase 2   3ezr          PF00069   9606   
1  2.7.11.22  Cyclin-dependent kinase 2   5osj          PF00069   9606   
2  2.7.11.22  Cyclin-dependent kinase 2   2r3p          PF00069   9606   
3  2.7.11.22  Cyclin-dependent kinase 2   2vtr          PF00069   9606   
4        NaN                  Cyclin-A2   1ogu  PF00134,PF02984   9606   

  uniprot_accession_best       e_value  percentage_identity  
0                 P24941  1.200000e-76                100.0  
1                 P24941  1.200000e-76                100.0  
2                 P24941  1.200000e-76                100.0  
3                 P24941  1.200000e-76                100.0  
4                 P20248  1.200000e-76                100.0  


We can save the results to a CSV file which we can load into excel

In [9]:
df.to_csv("search_results.csv")

There isn't a cut off of eValue or percentage identity in our search
so we should look what the values go to

we can select the column and find the minimum value with .min() or maximum value with .max()

In [11]:
df['percentage_identity'].max()

100.0

In [13]:
df['percentage_identity'].min()

36.1

same for e value - here we want the min and max


In [14]:
df['e_value'].min()

1.2e-76

In [15]:
df['e_value'].max()

5.6e-20

We can see that percentage identity drops to as low as 36%
Lets say we want to restrict it to 50%

In [16]:
df2 = df.query('percentage_identity > 50')

We stored the results in a new Dataframe called "df2"

In [17]:
df2.head()

   ec_number              molecule_name pdb_id   pfam_accession tax_id  \
0  2.7.11.22  Cyclin-dependent kinase 2   3ezr          PF00069   9606   
1  2.7.11.22  Cyclin-dependent kinase 2   5osj          PF00069   9606   
2  2.7.11.22  Cyclin-dependent kinase 2   2r3p          PF00069   9606   
3  2.7.11.22  Cyclin-dependent kinase 2   2vtr          PF00069   9606   
4        NaN                  Cyclin-A2   1ogu  PF00134,PF02984   9606   

  uniprot_accession_best       e_value  percentage_identity  
0                 P24941  1.200000e-76                100.0  
1                 P24941  1.200000e-76                100.0  
2                 P24941  1.200000e-76                100.0  
3                 P24941  1.200000e-76                100.0  
4                 P20248  1.200000e-76                100.0  
Number of entries in the Dataframe: 441
Max value of percentage identity: 100.0
Min value of percentage identity: 54.2


Number of entries in the Dataframe

In [18]:
len(df2)

441

Max value of percentage identity

In [20]:
df2['percentage_identity'].max()

100.0

Min value of percentage identity

In [19]:
df2['percentage_identity'].min()

54.2

How many unique Pfam domains or UniProts did we get back?

We can group the results by Pfram using "groupby" and then counting the results

In [12]:
df.groupby('pfam_accession').count()

,ec_number,molecule_name,pdb_id,tax_id,uniprot_accession_best,e_value,percentage_identity
pfam_accession,,,,,,,
PF00069,699,701,701,701,701,701,701
PF00134,0,2,2,2,2,2,2
"PF00134,PF02984",0,64,64,64,64,64,64
"PF00134,PF09241",0,3,3,3,3,3,3
"PF00183,PF02518",0,1,1,1,1,1,1
PF00564,1,1,1,1,1,1,1
PF01111,0,3,3,3,3,3,3
PF01335,0,3,3,3,3,3,3
PF02234,0,3,3,3,3,3,3


same for uniprot accession
This time we will sort the values by the number of PDB entries ("pdb_id"'s) they appear in.

In [16]:
df.groupby('uniprot_accession_best').count().sort_values('pdb_id', ascending=False)

,ec_number,molecule_name,pdb_id,pfam_accession,tax_id,e_value,percentage_identity
uniprot_accession_best,,,,,,,
P24941,339,339,339,339,339,339,339
P28482,102,102,102,102,102,102,102
Q16539,84,84,84,84,84,84,84
P63086,56,56,56,56,56,56,56
P20248,0,49,49,49,49,49,49
...,...,...,...,...,...,...,...
P49137,1,1,1,1,1,1,1
A9UJZ9,1,1,1,1,1,1,1
P52564,1,1,1,0,1,1,1


In this case the most common UniProt accession is P24941